https://github.com/pytorch/examples/blob/a38cbfc6f817d9015fc67a6309d3a0be9ff94ab6/mnist/main.py

In [22]:
%%bash
pip install torch

In [23]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self._i:int=0

    def forward(self, x):
        self._i=self._i+1
        if self._i % 100==0:
          print(f"i@forward={self._i}")
          print(f"[LOGG] type(x)={type(x)}, x.shape={x.shape}")
          #[LOGG] type(x)=<class 'torch.Tensor'>, x.shape=torch.Size([64, 1, 28, 28])

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [25]:
log_interval=40

def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [26]:


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [27]:
device = torch.device("cpu")
batch_size=64
test_batch_size=1000
epochs=2
lr=1.0
gamma=0.7

In [ ]:
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}


transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301955
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.385828
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.427437
i@forward=100
[LOGG] type(x)=<class 'torch.Tensor'>, x.shape=torch.Size([64, 1, 28, 28])
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.140896
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.356441
i@forward=200
[LOGG] type(x)=<class 'torch.Tensor'>, x.shape=torch.Size([64, 1, 28, 28])
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.183897
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.148908
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.117744
i@forward=300
[LOGG] type(x)=<class 'torch.Tensor'>, x.shape=torch.Size([64, 1, 28, 28])
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.093396
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.229910
i@forward=400
[LOGG] type(x)=<class 'torch.Tensor'>, x.shape=torch.Size([64, 1, 28, 28])
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.095298
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.122216
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.082576
i@fo